#### হাগিংফেসে লগিন হচ্ছে যাতে পরবর্তীতে মডেল সেখানে সংরক্ষণ করা যায়। এটি একটি ঐচ্ছিক ধাপ।

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## ডেটাসেট প্রস্তুতি

In [2]:
from datasets import load_dataset

In [3]:
datasets = load_dataset("text", data_files={"train": '../Datasets/wiki/train.txt', 
"validation": '../Datasets/wiki/val.txt'})

Using custom data configuration default-4d8e303c928b4712
Reusing dataset text (C:\Users\samin\.cache\huggingface\datasets\text\default-4d8e303c928b4712\0.0.0\acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08)
100%|██████████| 2/2 [00:00<00:00, 10.39it/s]


In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1967852
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 490947
    })
})

In [13]:
datasets["train"][10]

{'text': 'চর্যাপদ ভাষার আদি নিদর্শন'}

In [15]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(datasets["train"])

,text
0,বন্দি অবস্থায় তিনি ইরাকি সৈন্যদের দ্বারা গণধর্ষণের শিকার হন
1,এই গোষ্ঠীগুলির একটি হ্রাসকৃত কোয়েলম রয়েছে যাকে সিউডোকোয়েলম বলা হয়
2,জনক সীতার পিতা মাটি চাষ করার সময় তার লাঙ্গলের নীচে তিনি একটি শিশুকন্যা সীতা খুঁজে পান
3,বব টেলর টেস্টে খেললেও বেয়ারস্টো একদিনের খেলাগুলোয় নিয়মিতভাবে অংশ নেন
4,সোয়াজিল্যান্ডে ব্রিটিশ শাসন ১৯০৬ ১৯৬৮ ১৯০৩ সালে দ্বিতীয় বোয়ার যুদ্ধে ব্রিটিশদের বিজয়ের পর সোয়াজিল্যান্ড একটি ব্রিটিশ প্রোটেক্টরেট হয়ে ওঠে যা সোয়াজিল্যান্ড প্রোটেক্টরেট নামে পরিচিত
5,উদ্ভব অনেকের মতে আধুনিক স্বরবৃত্ত ছন্দ মধ্যযুগীয় কাব্য শ্রীকৃষ্ণকীর্তন এর ধামালি ছন্দ থেকে উদ্ভূত কেননা ধামালিকাব্য নামে পরিচিত ধামালি ছন্দে স্বরবৃত্তের পূর্ববর্তী রূপটি পরিলক্ষিত হয়
6,হিন্দু ধর্মে বিশ্বাস বা ধর্মের একীভূত বিশ্বাসের ঘোষিত কোন ব্যবস্থা নেই বরং এটি ভারতের ধর্মীয় বহুত্ববোধের বিশ্বাসের সমন্বিত একটি বিশাল শব্দ
7,সিপিআই এম নেতা হাসিম আব্দুল হালিম বিধানসভার অধ্যক্ষ নির্বাচিত হন এই পদটি তিনি ২০১১ সাল পর্যন্ত ধরে রেখেছিলেন
8,হৈলেবিডুর মন্দিরগুলি এখন প্রস্তাবিত ইউনেস্কো বিশ্ব ঐতিহ্যবাহী স্থান
9,ভাউপুর পাটোয়ারী বাড়ি জামে মসজিদ


#### মিনিএলএম তাদের টোকেনাইজার হিসেবে "xlm-roberta-base" ব্যবহার করে

In [4]:
model_checkpoint = "xlm-roberta-base"

#### যন্ত্রাংশের ক্ষমতার উপর নির্ভর করে ব্লক সাইজ নির্ধারণ করুন টোকেনাইজেশনের উদ্দেশে

In [6]:
block_size = 128

In [7]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
from transformers import Trainer, TrainingArguments

In [9]:
import math

## মাস্কড ল্যাংগুয়েজ মডেলিং

মাস্কড ল্যাংগুয়েজ মডেলিঙে এলোমেলো ভাবে কিছু শব্দ মাস্ক বা ঢেকে দেওয়া হয় এবং মডেলটি সেই শূন্যস্থান পুরণ করার চেষ্টা করে। এভাবে মডেলটি শিক্ষাগ্রহণ করে।

In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length')

In [22]:
tokenize_function(datasets["train"][:1])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

আবার ব্লকে বিভক্ত করা হচ্ছে ট্র্যাইনিং এর জন্য

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1024,
    num_proc=8,
)

স্মৃতিতে সংরক্ষণ করা হচ্ছে যাতে পরবর্তীতে এই সময় গ্রাসকারী পদক্ষেপটি আর না নিতে হয়

In [ ]:
lm_datasets.save_to_disk('../Datasets/wiki/lm_datasets_1024')

মাইক্রোসফটের মিনিএলএম মডেলটি হাগিংফেস থেকে আমদানি করা হচ্ছে

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('microsoft/Multilingual-MiniLM-L12-H384')

ট্র্যাইনিং এর জন্য বিভিন্ন বৈশিষ্ট্য পরিবর্তন করা হচ্ছে প্রয়োজন অনুসারে

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"Multilingual-MiniLM-L12-H384-MLM-finetuned-wikipedia_bn",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    save_steps=10000,
    num_train_epochs=3,
    resume_from_checkpoint=True,
    report_to="none"
)

আমাদের স্যাম্পলগুলোকে টেন্সর হিসেবে ব্যাচ করবো ডাটা কোলেটরের সাহায্যে

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

অবশেষে শিক্ষাদান করা হচ্ছে

In [ ]:
trainer.train()

হাগিংফেসে আপ্লোড করা হচ্ছে সর্বশেষ মডেলটি যাতে পরবর্তীতে সহজে ব্যাবহার করা যায়।

In [ ]:
trainer.push_to_hub()